## Import Libraries

In [ ]:
from google.colab import drive
import os
import shutil
import numpy as np
import random as rn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt
from sklearn import metrics

## Utilities

In [ ]:
dataset_path = "/content/drive/MyDrive/BrainTumorDataset"

def set_seed ():
	''' 
	set_seed is used to obtain reproducible results using keras during the development phase
	'''
	seed = 10
	# The below is necessary for reproducible results of certain Python hash-based operations.
	os.environ["PYTHONHASHSEED"]="0"
	# The below is necessary for starting Numpy generated random numbers in a well-defined initial state.
	np.random.seed(seed)
	# The below is necessary for starting core Python generated random numbers in a well-defined state.
	rn.seed(seed)
	# The below tf.random.set_seed will make x number generation in TensorFlow have a well-defined initial state.
	tf.random.set_seed(seed)          

def dataset_load():
    '''
    load dataset from folder, shuffle it and categorize labels from 0 to 3 (4 classes)
    '''
    labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
    image_size = 250
    X_dataset = []
    y_dataset = []
    for lb in labels:
        folderPath = os.path.join('/content/drive/MyDrive/BrainTumorDataset/Preprocessed/Unified', lb)
        for j in tqdm(os.listdir(folderPath)):
            img = cv2.imread(os.path.join(folderPath,j))
            img = cv2.resize(img,(image_size, image_size))
            X_dataset.append(img)
            y_dataset.append(lb)

    X_dataset = np.array(X_dataset)
    y_dataset = np.array(y_dataset)

    X_dataset, y_dataset = shuffle(X_dataset, y_dataset, random_state=42)

    y_dataset_new = []
    for i in y_dataset:
       y_dataset_new.append(labels.index(i))
    y_dataset = y_dataset_new
    y_dataset = tf.keras.utils.to_categorical(y_dataset,4)

    return X_dataset, y_dataset

def show_training_and_validation_performance(history):
	'''
	show_training_and_validation_performance is used to plot the performances during the training phase
	:param history: object in which are recorded all the events
	'''
	acc = history.history['accuracy']
	val_acc = history.history['val_accuracy']
	loss = history.history['loss']
	val_loss = history.history['val_loss']

	epochs = range(len(acc))

	plt.plot(epochs, acc, 'bo', label='Training accuracy')
	plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
	plt.title('Training and validation accuracy')
	plt.legend()

	plt.figure()

	plt.plot(epochs, loss, 'bo', label='Training loss')
	plt.plot(epochs, val_loss, 'b', label='Validation loss')
	plt.title('Training and validation loss')
	plt.legend()

	plt.show()

def compile_model (model, optimizer='adam', learning_rate = 0.001):
	'''
	compile_model is used to compile the current model
	:param model: model to compile
	:param optimizer: optimizer to be used
	:param learning_rate: learning rate parameter for the optimizer
	'''
	if optimizer == 'adam':
		model.compile(loss="binary_crossentropy",
		optimizer=optimizers.Adam(learning_rate=learning_rate),
		metrics=["accuracy"])
	elif optimizer == 'rmsprop':
		model.compile(loss="binary_crossentropy",
					optimizer = optimizers.RMSprop(learning_rate=learning_rate),
					metrics=['accuracy'])
	model.summary()

def run_model (model, model_name, epochs = 20, patience=5, monitor='val_loss'):
	'''
	run_model is used to run the current mode
	:param model: model to run
	:param model_name: name given to save the model
	:param epochs: how many epochs to do
	:param patience: patience value for Early Stopping
	:param monitor: what to monitor for Early Stopping and Model Checkpoint
	'''
	# local save path for the models
	save_path = dataset_path + '/' + model_name + '.h5'
	callbacks_list = [
					keras.callbacks.EarlyStopping(monitor=monitor, patience=patience),
					keras.callbacks.ModelCheckpoint(
						filepath = save_path,
						monitor=monitor,
						verbose=1,
						save_best_only=True)
					]
	history = model.fit(train_dataset,
						epochs=epochs,
						validation_data=validation_dataset,
						callbacks=callbacks_list)
	# save on Drive only the best model
	shutil.copy(save_path, dataset_path + '/' + model_name + '.h5')
	show_training_and_validation_performance(history)

from sklearn.metrics import roc_curve, auc, roc_auc_score
from itertools import cycle

def plot_roc_curve(y_test, y_pred):
	'''
	plot_roc_curve is used to plot the ROC curve
	:param y_test: true labels
	:param y_pred: predicted labels
	'''
	labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
	fpr = dict()
	tpr = dict()
	roc_auc = dict()
	for i in range(4):
		fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
		roc_auc[i] = auc(fpr[i], tpr[i])


	colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green'])
	for i, color in zip(range(4), colors):
		plt.plot(
			fpr[i],
			tpr[i],
			color=color,
			lw=2,
			label='ROC curve of class {0} (area = {1:0.2f})'.format(labels[i], roc_auc[i]),
		)

	plt.plot([0, 1], [0, 1], 'k--', lw=2)
	plt.xlim([0.0, 1.0])
	plt.ylim([0.0, 1.05])
	plt.xlabel('False Positive Rate')
	plt.ylabel('True Positive Rate')
	plt.title('Some extension of Receiver operating characteristic to multiclass')
	plt.legend(loc='lower right')
	plt.show()

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(y_test, y_pred):
	labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
	y_pred = np.argmax(y_pred, axis=1)
	y_test = np.argmax(y_test, axis=1)
	cm = confusion_matrix(y_test, y_pred, labels=labels)
	disp = ConfusionMatrixDisplay(confusion_matrix=cm)#, display_labels=['0', '1'])
	disp.plot()
	plt.show()

def evaluate_model (model, test_dataset):
	'''
	evaluate_model is used to plot some statistics about the performance on the test set
	:param model: model to consider
	'''
	y_score = model.predict(test_dataset)
	y_pred = np.rint(y_score) # to have 0 or 1
	y_true = tf.concat([labels_batch for data_batch, labels_batch in test_dataset], axis = 0)
	print("Classification report: ")
	print(metrics.classification_report(y_true,y_pred,digits = 4))
	metrics.ConfusionMatrixDisplay.from_predictions(y_true, y_pred)
	plot_roc_curve(y_true,y_pred)
	confusion_matrix(y_true,y_pred)

## Load Dataset